# Lesson 29: JavaScript for stand-alone Bokeh apps

<hr>

In [1]:
import pandas as pd
import numpy as np
import scipy.stats

import bokeh.plotting
import bokeh.io

bokeh.io.output_notebook()

Loading BokehJS ...

<hr>

As we have seen when [building dashboards](l25_dashboards.ipynb), we can build remarkable interactivity in our data displays, allowing for low-effort exploration of data sets. Bokeh allows us to link plots to Python code that can update our displays as we adjust parameters or select variables. We can also serve up apps to get only the data displays we like on their own browser tabs. I hope you recognize how powerful this is for exploring your data sets.

There is a major drawback to this approach, though. In order to interactively explore using the dashboard, you need to have a Python interpreter running, either in the Jupyter notebook or when you serve your dashboards using `bokeh serve` on the command line. If you wanted to include an interactive plot in a publication, it would be nice to be able to interact with the plots/dashboard directly in the browser without the reader having to launch a Jupyter notebook or serve it up on their local machine (and have all of the necessary installations). In some cases, you will need the Python interpreter for sophisticated dashboards that do involved calculations. But in many cases, with just a little JavaScript, you can make beautiful interactive graphics rendered in pure HTML with JavaScript.

(Note, though, that in coming years, [PyScript](https://pyscript.net) may change the game on this.)

## JavaScript and Bokeh

We have been using Bokeh for rendering out plots and dashboards. As we have seen, Bokeh is a Python library for generating graphics that can be visualized and interacted with in the browser. In order to display the graphics and enable the interactions, Bokeh relies on its client-side [JavaScript](https://en.wikipedia.org/wiki/JavaScript) library, [BokehJS](https://docs.bokeh.org/en/latest/docs/dev_guide/bokehjs.html). It is this JavaScript library that handles all of the rendering, zooming, selecting, etc., as well has the widgets. The callback functions that get executed to update information about the plot based on changes in the widgets are in Python. This is what requires the Python interpreter to be running.

Conveniently, Bokeh allows for pure JavaScript callbacks. And it turns out that if you know just a little JavaScript, you can make your dashboards have full functionality standing alone in the browser without Python running.

So, in this lesson, we will learn a little JavaScript to make our dashboards be totally portable. We will not delve into the details of the language (that would require a complete bootcamp!), but will just give a little syntax. We will do it all in the context of making interactive plots. As you will see, a just a little JavaScript goes a long way!

## Example 1: Exploring the Normal pdf

In our [lesson on dashboarding](l25_dashboards.ipynb), we build an interactive plot of the probability density function (pdf) for the Normal distribution. We used the `scipy.stats` package to get the mathematical expression for the pdf. Since we will code it up in JavaScript, I write it here.

\begin{align}
f(x;\mu, \sigma) = \frac{\mathrm{e}^{-(x-\mu)^2/2\sigma^2}}{\sqrt{2\pi\sigma^2}}.
\end{align}

I will not recreate the little dashboard we made in the dashboarding lesson for visualizing the Normal pdf, but so that it can stand alone. As a reminder, here is the code (sans imports) we used before to create the dashboard.

```python
# Parameters; we'll start with standard Normal
mu = 0.0
sigma = 1.0

# Generate data
x = np.linspace(-10, 10, 200)
pdf = scipy.stats.norm.pdf(x, loc=mu, scale=sigma)

# Column data source for plot
source = bokeh.models.ColumnDataSource(dict(x=x, pdf=pdf))

# Build figure
p = bokeh.plotting.figure(
    frame_width=350,
    frame_height=200,
    x_axis_label='x',
    y_axis_label='f(x)',
    x_range=[-10, 10],
)

# Put line on plot
p.line(source=source, x='x', y='pdf', line_width=2)

# Sliders
mu_slider = bokeh.models.Slider(title="µ", start=-5.0, end=5.0, step=0.1, value=0.0, width=100)
sigma_slider = bokeh.models.Slider(title="σ", start=0.1, end=5.0, step=0.1, value=1.0, width=100)

def norm_callback(attr, old, new):
    """Callback for updating data in Normal PDF plot."""
    # Pull the values off of each slider
    mu = mu_slider.value
    sigma = sigma_slider.value
    
    # Re-compute the y-values
    pdf = scipy.stats.norm.pdf(source.data['x'], loc=mu, scale=sigma)
    
    # Update the column data source
    source.data["pdf"] = pdf


# Connect sliders
mu_slider.on_change('value', norm_callback)
sigma_slider.on_change('value', norm_callback)

# Put the sliders one on top of the other
slider_layout = bokeh.layouts.column(
    bokeh.layouts.Spacer(height=30),
    mu_slider,
    bokeh.layouts.Spacer(height=15),
    sigma_slider,    
)

# Put the sliders to the right of the plot
norm_layout = bokeh.layouts.row(
    p,
    bokeh.layouts.Spacer(width=15),
    slider_layout
)


def norm_app(doc):
    doc.add_root(norm_layout)

    
bokeh.io.show(norm_app, notebook_url=notebook_url)   
```

Now, let's build the dashboard with a JavaScript callback so it can stand alone with a Python interpreter running. First, the setup of the setup of the sliders, data, and figure are the same as we would normally do.

In [2]:
# Initial setup of the sliders is the same
mu_slider = bokeh.models.Slider(title="µ", start=-5, end=5, step=0.1, value=0)
sigma_slider = bokeh.models.Slider(title="σ", start=0.1, end=5, step=0.1, value=1)

# Setup of x and y values is also the same
x = np.linspace(-10, 10, 200)
pdf = scipy.stats.norm.pdf(x, loc=0, scale=1)

# Set up the plot as a Bokeh figure
p = bokeh.plotting.figure(
    frame_width=300,
    frame_height=200,
    x_axis_label="x",
    y_axis_label="f(x; μ, σ)",
    x_range=[-10, 10],
)

Next, we set up a `ColumnDataSource` when we build the plot so we have access to it when manipulate the sliders and use a JavaScript callback.

In [3]:
# Use a ColumnDataSource
source = bokeh.models.ColumnDataSource(dict(x=x, pdf=pdf))
p.line(source=source, x="x", y="pdf", line_width=2);

Laying out the sliders and plot is the same as before.

In [4]:
# Laying out the widgets is the same
slider_layout = bokeh.layouts.column(
    bokeh.layouts.Spacer(height=30),
    mu_slider,
    bokeh.layouts.Spacer(height=15),
    sigma_slider,
    width=200,
)

# Lay out with the plot and widgets
norm_layout = bokeh.layouts.row(p, bokeh.layouts.Spacer(width=15), slider_layout)

Now, we need to write JavaScript code for the callback. Whenever a slider changes value, we want to recompute the values of the Normal pdf. Here is JavaScript code to do that.

```javascript
function normalPDF(x, mu, sigma) {
    let expArg = (x - mu) ** 2 / 2.0 / sigma ** 2;
    return Math.exp(-expArg) / sigma / Math.sqrt(2 * Math.PI);
}

// For convenience, get views into the ColumnDataSource data
let x = source.data['x'];
let pdf = source.data['pdf'];

// Pull the values off of the sliders
let mu = mu_slider.value;
let sigma = sigma_slider.value;

// Loop through and update
let xLen = x.length;
for (let i = 0; i < xLen; i++) {
    pdf[i] = normalPDF(x[i], mu, sigma);
}

// Once the ColumnDataSource is changed, we need to use its 
// .change.emit() method to update the plot
source.change.emit();
```

Let's discuss the syntax. First, comments in JavaScript are preceded with `//`. Functions in JavaScript are declared like this:

```javascript
function funcName() {
    // function body
}
```

The `function` keyword is followed by the function name, with its arguments in parentheses separated by commas. The function body is then contained between braces.

Within the `normalPDF()` function, we make some calculations. First, we define a variable that is the argument for the exponentiation, $-(x-\mu)^2/2\sigma^2$. Note that whenever we use a new variable, it must be preceded by the keyword `let` (or `var`, but we will not get into the differences between those keywords). The arithmetic operators, including raise-to-power, are the same as in Python. We end each statement with a semicolon.

Next comes the `return` statement. Its syntax is just like Python. The only difference in the return statement is that we need to use JavaScript's `Math` module to get access to the exponentiation and square root functions, and also to the value of π. Note that this function is _not_ vectorized. It assumes `x`, `mu`, and `sigma` are scalar valued and computes the value of the Normal pdf at that given value of `x`.

Conveniently, accessing the `ColumnDataSource.data` attribute using JavaScript is exactly as in Python. For convenience in the above code, we create views into the `ColumnDataSource` using `let x = source.data['x'];` and `let pdf = source.data['pdf'];`.

Next, we pull the values from the sliders. This is also done as in Python (except with the necessary `let`).

Now that we have access to the data and the slider values, we need to loop through the data in the `ColumnDataSource` and update it according to the values of the sliders. A `for` loop in JavaScript typically has the following form.

```javascript
for (let i = 0; i < lastValueOfiYouWant; i++) {
    // body of for loop
}
```

Inside the parenthetical after the `for` keyword are two semicolons. Before the first is an initialization. After the first semicolon is a conditional statement. The loop will continue so long as that conditional statement gives `true` (in JavaScript the keyword for truth is all lowercase, unlike in Python). Finally, after the second semicolon is the iteration statement. It is executed whenever a trip through the body of the loop concludes. The syntax `i++` is equivalent to Python's `i += 1`.

So, the `for` loop in the above statement loops through each x-value in the plot and updates the value of the Normal pdf using the parameter values we have pulled off of the sliders.

Finally, in order for the plot to update, you need to execute the `source.change.emit()` function, which pushes the updates of the `ColumnDataSource` to the rendered plot.

This code is in JavaScript, so obviously the Python interpreter will not know what to do with it. It is not really used until things get handed off to the BokehJS library. So, we need to store the code as a Python string.

In [5]:
# JavaScript code for the callback stored as a string
jscode = """
function normalPDF(x, mu, sigma) {
    let expArg = (x - mu) ** 2 / 2.0 / sigma ** 2;
    return Math.exp(-expArg) / sigma / Math.sqrt(2 * Math.PI);
}

// For convenience, get views into the ColumnDataSource data
let x = source.data['x'];
let pdf = source.data['pdf'];

// Pull the values off of the sliders
let mu = mu_slider.value;
let sigma = sigma_slider.value;

// Loop through and update
let xLen = x.length;
for (let i = 0; i < xLen; i++) {
    pdf[i] = normalPDF(x[i], mu, sigma);
}

// Once the ColumnDataSource is changed, we need to use its 
// .change.emit() method to update the plot
source.change.emit();
"""

Now that we have the code, we need to connect it to the sliders. That is, we need to let the slider objects know that when they change, the JavaScript code should be triggered.

In [6]:
args = dict(source=source, mu_slider=mu_slider, sigma_slider=sigma_slider)

# Connect the callback to the sliders
mu_slider.js_on_change("value", bokeh.models.CustomJS(code=jscode, args=args))
sigma_slider.js_on_change("value", bokeh.models.CustomJS(code=jscode, args=args))

The `js_on_change()` method links the sliders to the JavaScript code. The first argument of the `js_on_change()` method is what is changing that will trigger the callback. In this case, we want the callback to be triggered whenever the value of the slider changes. The second argument is a Bokeh `CustomJS` object. This is constructed by specifying the code as a string and the arguments that will be passed to the JavaScript code (in this case the `ColumnDataSource` and the two sliders).

Note that if we used Panel-based sliders instead of the Bokeh sliders we are using here, the syntax is

```python
slider_created_using_panel.jscallback(value=jscode, args=args)
```

Now we are ready to check out our layout! We only need to call `bokeh.io.show()` and do not need to supply a `notebook_url` argument because there are no Python callbacks.

In [7]:
# Display the layout
bokeh.io.show(norm_layout)

This is more or less the same about of work we did creating the dashboard as in the previous lesson except the callbacks were written in JavaScript and linked using `js_on_change()`.

Writing callbacks in a new language is not trivial. But, I hope you can see that it's not _too_ bad to code up the JavaScript. The payoff is big. You get a stand-alone interactive plot that _anyone_ can view and play with in their browser without even having Python installed.

Let's now make that HTML document.

In [8]:
bokeh.io.save(norm_layout, "normal_pdf_app.html", resources=bokeh.resources.CDN, title="Normal PDF")

'/Users/bois/Dropbox/git/programming_bootcamp/2024/content/lessons/normal_pdf_app.html'

We now have our stand-alone app!

## Exploring a data set

In the [dashboarding lesson](l25_dashboards.ipynb), we used data from the [Beattie, et al.](https://doi.org/10.1098/rsos.160321) study of how sleep deprivation affects facial matching ability to build a dashboard to explore a data set. Here, I will demonstrate how to build the same dashboard, as a standalone app with JavaScript callbacks. Such a plot could appear in a publication with full interactivity for all readers.

Of course, we'll start with loading in the data.

In [9]:
df = pd.read_csv('data/gfmt_sleep.csv', na_values='*')

# Add column for insomnia
df['insomnia'] = df['sci'] <= 16

df.head()

,participant number,gender,age,correct hit percentage,correct reject percentage,percent correct,confidence when correct hit,confidence when incorrect hit,confidence when correct reject,confidence when incorrect reject,confidence when correct,confidence when incorrect,sci,psqi,ess,insomnia
0,8,f,39,65,80,72.5,91.0,90.0,93.0,83.5,93.0,90.0,9,13,2,True
1,16,m,42,90,90,90.0,75.5,55.5,70.5,50.0,75.0,50.0,4,11,7,True
2,18,f,31,90,95,92.5,89.5,90.0,86.0,81.0,89.0,88.0,10,9,3,True
3,22,f,35,100,75,87.5,89.5,NaN,71.0,80.0,88.0,80.0,13,8,20,True
4,27,f,74,60,65,62.5,68.5,49.0,61.0,49.0,65.0,49.0,13,9,12,True


To build the dashboard, we will start by setting up the selectors, one for the x-axis variable, one for the y-axis variable, and one for how we will color the glyphs. Again, this is the same code as before.

In [10]:
# Options for x- and y- selector; omit part. num., gender, and insomnia
xy_options = list(
    df.columns[~df.columns.isin(["participant number", "gender", "insomnia"])]
)

x_selector = bokeh.models.Select(
    title="x", options=xy_options, value="percent correct", width=200,
)

y_selector = bokeh.models.Select(
    title="y", options=xy_options, value="confidence when correct", width=200,
)

colorby_selector = bokeh.models.Select(
    title="color by", options=["none", "gender", "insomnia",], value="none", width=200,
)

Next, we can set up the `ColumnDataSource`. When we built the dashboard in the previous lesson, we had a single ColumnDataSource that got updated by pulling out of the data frame in the Python callbacks. Now, we will be using only JavaScript callbacks, and JavaScript does not know what a Pandas data frame is. Therefore, we create a *second* ColumnDataSource from the data frame that we can pull from to populate the ColumnDataSource that is used in the rendering of the plot.

First, we make the source that is used in rendering. It is just as before.

In [11]:
render_source = bokeh.models.ColumnDataSource(
    {
        "x": df["percent correct"],
        "y": df["confidence when correct"],
        "color": ["#1f77b3"] * len(df),
    }
)

Now that we have the `ColumnDataSource` for rendering, we need to convert the data frame `df` into a `ColumnDataSource` so we can have access to it using JavaScript.

In [12]:
source = bokeh.models.ColumnDataSource(df)

Now we can make our figure!

In [13]:
p = bokeh.plotting.figure(
    frame_height=250,
    frame_width=250,
    x_axis_label="percent correct",
    y_axis_label="confidence when correct",
)

circle = p.scatter(
    source=render_source, x="x", y="y", fill_color="color", line_color="color",
)

Finally, we write our JavaScript code for the callbacks. These will happen whenever the selectors change. Following is the JavaScript code.

```javascript
// Extract what we want to color by from selector
let colorby = colorby_selector.value;

// View of the colors for convenience
let colors = render_source.data['color'];

// Convenient to have the number of data points
let n = colors.length;

// Update rendered data from full data set
render_source.data['x'] = source.data[x_selector.value];
render_source.data['y'] = source.data[y_selector.value];

// Update colors
if (colorby === 'none') {
    for (let i = 0; i < n; i++) {
        colors[i] = '#1f77b3';
    }
}
else if (colorby === 'gender') {
    for (let i = 0; i < n; i++) {
        if (source.data['gender'][i] === 'f') {
            colors[i] = '#1f77b3';
        }
        else {
            colors[i] = '#ff7e0e';
        }
    }
}
else if (colorby === 'insomnia') {
    for (let i = 0; i < n; i++) {
        if (source.data['insomnia'][i]) {
            colors[i] = '#1f77b3';
        }
        else {
            colors[i] = '#ff7e0e';
        }
    }
}

// Update axis labels to reflect what was selected
xaxis[0].axis_label = x_selector.value;
yaxis[0].axis_label = y_selector.value;

render_source.change.emit();
```

Except for the `if` statements, which have new syntax, the code should be self-explanatory. There is the added wrinkle of the `xaxis` and `yaxis` variables, which we will have to pass into the JavaScript code as arguments. These are Bokeh `LinearAxis` objects that we need to access to update the axis labels for what we are plotting.

In JavaScript, an `if`-`else` statement has the structure

```c
if (statement_returning_boolean) {
    // code to be executed if statement_returning_boolean is true
}
else if (another_statement_returning_boolean) {
    // code to be executed if another_statement_returning_boolean is true
}
else {
    // code to be executed if all booleans were false
}
```

Note the use of `===` for the equals comparison operator. In JavaScript, if I use `a == b`, JavaScript tries to do type comparisons to compare things. `0 == ''` will evaluate `true`. The `===` operator means that there are no type comparisons. So, `0 === ''` evaluates `false`.

Now that we have our JavaScript code, let's put it to use!

In [14]:
jscode = """
// Extract what we want to color by from selector
let colorby = colorby_selector.value;

// View of the colors for convenience
let colors = render_source.data['color'];

// Convenient to have the number of data points
let n = colors.length;

// New data
render_source.data['x'] = source.data[x_selector.value];
render_source.data['y'] = source.data[y_selector.value];

// Update colors
if (colorby === 'none') {
    for (let i = 0; i < n; i++) {
        colors[i] = '#1f77b3';
    }
}
else if (colorby === 'gender') {
    for (let i = 0; i < n; i++) {
        if (source.data['gender'][i] === 'f') {
            colors[i] = '#1f77b3';
        }
        else {
            colors[i] = '#ff7e0e';
        }
    }
}
else if (colorby === 'insomnia') {
    for (let i = 0; i < n; i++) {
        if (source.data['insomnia'][i]) {
            colors[i] = '#1f77b3';
        }
        else {
            colors[i] = '#ff7e0e';
        }
    }
}

// Update axis labels to reflect what was selected
xaxis[0].axis_label = x_selector.value;
yaxis[0].axis_label = y_selector.value;

render_source.change.emit();
"""

args = dict(
    render_source=render_source,
    source=source,
    colorby_selector=colorby_selector,
    x_selector=x_selector,
    y_selector=y_selector,
    xaxis=p.xaxis,
    yaxis=p.yaxis,
)

Note that I have put both data sources, the selectors, and the `p.axis` and `p.yaxis` objects as arguments to the JavaScript code. Now we can link the callbacks.

In [15]:
colorby_selector.js_on_change("value", bokeh.models.CustomJS(code=jscode, args=args))
x_selector.js_on_change("value", bokeh.models.CustomJS(code=jscode, args=args))
y_selector.js_on_change("value", bokeh.models.CustomJS(code=jscode, args=args))

And finally, let's lay it out!

In [16]:
layout = bokeh.layouts.row(
    p,
    bokeh.layouts.Spacer(width=15),
    bokeh.layouts.column(
        x_selector,
        bokeh.layouts.Spacer(height=15),
        y_selector,
        bokeh.layouts.Spacer(height=15),
        colorby_selector,
    ),
)

bokeh.io.show(layout)

Beautiful! We can again save the result and have a stand-alone interactive HTML file.

In [17]:
bokeh.io.save(
    layout,
    "sleep_facial_recognition.html",
    resources=bokeh.resources.CDN,
    title="sleep facial recognition",
)

'/Users/bois/Dropbox/git/programming_bootcamp/2024/content/lessons/sleep_facial_recognition.html'

## Conclusions

I hope you can see that a little JavaScript goes a long way! But knowing just some basic JavaScript syntax, you can build totally sharable, browser-display-and-playable data displays.

## Computing environment

In [18]:
%load_ext watermark
%watermark -v -p numpy,pandas,bokeh,jupyterlab

Python implementation: CPython
Python version       : 3.11.9
IPython version      : 8.20.0

numpy     : 1.26.4
pandas    : 2.2.1
bokeh     : 3.4.0
jupyterlab: 4.0.13

